# Reboot - SQL Advanced

Tonight we will work with the `blog.sqlite` database is available at this URL:  
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite`

In [7]:
!curl -L https://wagon-public-datasets.s3.amazonaws.com/sql_databases/blog.sqlite > data/blog.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  288k  100  288k    0     0   186k      0  0:00:01  0:00:01 --:--:--  186k


Let's have a look at our directory structure

In [8]:
!tree

.
├── README.md
├── books.csv
├── data
│   └── blog.sqlite
└── recap.ipynb

2 directories, 4 files


## 1. Schema ERD

❓ Open the `data/blog.sqlite` in DBeaver, explore the schema and draw it on [kitt.lewagon.com/db](https://kitt.lewagon.com/db).

_TODO: Double click this cell and **paste** a screenshot of the schema for future reference_.

---
## 2. Most liked posts

Complete the code to get **the 3 most liked posts**:

In [10]:
import sqlite3

conn = sqlite3.connect("data/blog.sqlite")
c = conn.cursor()

# TODO: write the query
query = """
    select p.title,count(l.id) as c
    from posts p 
    join likes l on p.id = l.post_id
    group by p.id 
    order by c desc
    limit 3
"""

# TODO: Execute the query
c.execute(query)

# TODO: Fetch and print the results
data = c.fetchall()
data

[('Half imagine another.', 84),
 ('Side foot leader popular.', 82),
 ('Area paper whatever mean.', 81)]

---

### Pretty Print using _pandas_

The readbility of our `print()` statements is not so good.

Next week, we will introduce [pandas](https://pandas.pydata.org/) which will vastly improve the UX of our data exploration in Notebooks.

Execute the following cell to load `pandas` library:

In [11]:
import pandas as pd

Then try the previous `query` again, delegating the job of fetching results + displaying them to the `read_sql_query` function of `pandas`:

In [12]:
pd.read_sql_query(query, conn)

,title,c
0,Half imagine another.,84
1,Side foot leader popular.,82
2,Area paper whatever mean.,81


---
## 3. Find the three users who 'liked' the most

In [13]:
query = """
select 
	u.first_name || " " || u.last_name as full_name, 
	count(l.id) as likes
from users u
join likes l on u.id = l.user_id 
GROUP BY u.id 
Order BY likes DESC 
LIMIT 3
"""

pd.read_sql_query(query, conn)

,full_name,likes
0,Michael Allen,236
1,Donna Ramirez,233
2,Hayley Williams,227


---
## 4. Find the most liked author

In [14]:
query = """
SELECT 
	users.first_name, 
	COUNT(likes.id) as like_count
FROM users  
JOIN posts ON users.id = posts.user_id 
JOIN likes ON posts.id = likes.post_id
GROUP BY users.id 
ORDER BY like_count DESC
LIMIT 1
"""

pd.read_sql_query(query, conn)

,first_name,like_count
0,Teresa,647


---
## 5. Who are the authors of the 3 most liked posts?

In [15]:
query = """
SELECT 
	COUNT(l.id) as total_likes,
	first_name || " " || last_name as full_name,
	p.title
FROM users u 
JOIN posts p ON u.id = p.user_id 
JOIN likes l ON p.id = l.post_id
GROUP BY p.id
ORDER BY total_likes DESC
LIMIT 3
"""

pd.read_sql_query(query, conn)

,total_likes,full_name,title
0,84,Melissa Henry,Half imagine another.
1,82,Cynthia Raymond,Side foot leader popular.
2,81,Alexander Cook,Area paper whatever mean.


---
## 6. How many people liked at least one post?

In [25]:
query = """
WITH user_likes AS (
    SELECT *
    FROM likes
    GROUP BY user_id
)
SELECT COUNT(*) 
FROM user_likes
"""

query = """
    SELECT COUNT( DISTINCT user_id)
    FROM likes
"""

pd.read_sql_query(query, conn)

,COUNT( DISTINCT user_id)
0,49


---
## 7. Compute the cumulative number of likes per day

In [34]:
query = """
    SELECT 
        likes.created_at,
        SUM(likes.id) OVER (
            ORDER BY likes.created_at
        ) as cumul_likes
    FROM likes
    GROUP BY likes.created_at
"""

pd.read_sql_query(query, conn)

,created_at,cumul_likes
0,2019-01-01,837
1,2019-01-02,1013
2,2019-01-03,1691
3,2019-01-04,1845
4,2019-01-05,2052
...,...,...
331,2019-12-24,108378
332,2019-12-25,108743
333,2019-12-26,109706
334,2019-12-27,110418


---
## 8. (Optional) Who's the biggest fan/ fans of each author?

The biggest fan/ fans of an author is defined as the user or users who liked the author's posts the most. i.e. if there is a tie between fans that both liked an author 20 times, both fans should be returned alongside their like count and the author in question.
<br><br>
<details>
    <summary>💡 Click for Hint</summary>
    You might need to use <code>WITH</code>
</details>


In [ ]:
query = """
    
"""

pd.read_sql_query(query, conn)